In [16]:
import requests
import json

In [2]:
def deal_json_invaild(data):
    data = data.replace(' ','')
    data = data.replace("\n", "\\n").replace("\r", "\\r").replace("\n\r", "\\n\\r") \
        .replace("\r\n", "\\r\\n") \
        .replace("\t", "\\t")
    data = data.replace('":"', '&&testPassword1&&')\
        .replace('":', '&&testPassword2&&')\
        .replace('","', "$$testPassword$$")\
        .replace('{"', "@@testPassword@@")\
        .replace('"}', "**testPassword**")
    # print(data)

    data = data.replace('"', r'\"')\
        .replace('&&testPassword1&&', '":"').replace('&&testPassword2&&','":').replace('$$testPassword$$', '","').replace('@@testPassword@@', '{"').replace('**testPassword**', '"}')
    # print(data)
    return data

In [4]:
import oss2
ali_user_name = 'LTAI5tRAbpW9MNjDDnDAVCsq'
ali_pwd = '7RguIff46JiTrh43HkULgbXZPiIVqu'
auth = oss2.Auth(ali_user_name, ali_pwd)
# yourEndpoint填写Bucket所在地域对应的Endpoint。以华东1（杭州）为例，Endpoint填写为https://oss-cn-hangzhou.aliyuncs.com。
# 填写Bucket名称。
bucket = oss2.Bucket(auth, 'oss-ap-southeast-1.aliyuncs.com', 'aistormy2023')
oss_url_prefix = "https://aistormy2023.oss-ap-southeast-1.aliyuncs.com/"

file_name='test_cv.pdf'
content = open(file_name, 'rb').read()
ret = bucket.put_object(file_name, content)

# v2.0

In [19]:
url = 'http://114.248.215.173:32040'
api_list = [
"/test",
"/recruit/job/register",
"/recruit/job/query",
"/recruit/account/register",
"/recruit/account/query",
"/recruit/account/task/fetch",
"/recruit/account/task/report",
"/recruit/candidate/filter",
"/recruit/candidate/chat",
"/recruit/candidate/result"
]

In [20]:
api_dict = {}
for api in api_list:
    sp = api[1:].split('/')
    api_key = sp[-1]
    if len(sp)>1:
        api_key+=f'_{sp[-2]}'
    api_dict[api_key]=api
    print(f'{api_key}\t\t\t{api}')

test			/test
register_job			/recruit/job/register
query_job			/recruit/job/query
register_account			/recruit/account/register
query_account			/recruit/account/query
fetch_task			/recruit/account/task/fetch
report_task			/recruit/account/task/report
filter_candidate			/recruit/candidate/filter
chat_candidate			/recruit/candidate/chat
result_candidate			/recruit/candidate/result


## job op

In [43]:
r = requests.post(url+api_dict['register_job'],json={
    'platformType': 'Boss',
    'platformID': 'minsheng-kefu-manual-id',
    'jobName': '民生银行-外呼客服',
    'jd': '''银行岗位jd
招聘公司：知名大型商业银行
岗位名称：外呼客服
岗位职责：通过银行外呼系统对已经使用信用卡产生消费账单的用户，让用户办理分期还款，减轻还款压力。
岗位要求：年龄18-35周岁，大专及以上学历，普通话标准，良好的沟通表达能力
薪资福利：综合薪资8000-20000，上不封顶，入职缴纳北京五险一金，享受法定节假日
工作时间：早9晚6''',
    'robotApi': '/vision/chat/receive/message/minsheng/v1'
})
r.text

'{"ret": 0, "msg": "success", "data": {"jobID": "job_Boss_minsheng-kefu-manual-id"}}'

In [44]:
r = requests.post(url+api_dict['query_job'],json={
    'platformType': 'Boss',
    'platformID': 'minsheng-kefu-manual-id'
})
r.text

'{"ret": 0, "msg": "success", "data": {"jobID": "job_Boss_minsheng-kefu-manual-id"}}'

In [45]:
job_id="job_Boss_minsheng-kefu-manual-id"

## account op

In [46]:
r = requests.post(url+api_dict['register_account'],json={
    'platformType': 'Boss',
    'platformID': '27175761',
    'jobs': [job_id]
})
r.text

'{"ret": 0, "msg": "success", "data": {"accountID": "account_Boss_27175761"}}'

In [47]:
r = requests.post(url+api_dict['query_account'],json={
    'platformType': 'Boss',
    'platformID': '27175761',
})
r.text

'{"ret": 0, "msg": "success", "data": {"accountID": "account_Boss_27175761"}}'

In [48]:
account_id="account_Boss_27175761"

## task op

In [49]:
r = requests.post(url+api_dict['fetch_task'],json={
    'accountID': account_id,
})
r.text

'{"ret": 0, "msg": "success", "data": {"task": [{"helloSum": 50, "taskType": "batchTouch", "timeMount": [{"time": "09:00", "mount": 25}, {"time": "16:00", "mount": 25}], "jobID": "job_Boss_minsheng-kefu-manual-id"}]}}'

In [31]:
r = requests.post(url+api_dict['report_task'],json={
    'accountID': account_id,
    'taskStatus': [{
        'taskType': 'batchTouch',
        'details': {
            'candidateList': ['mock-589791359']
        }
    }]
})
r.text

'{"ret": 0, "msg": "success", "data": {"status": "ok"}}'

In [24]:
r = requests.post(url+api_dict['fetch_task'],json={
    'accountID': account_id,
})
r.text

'{"ret": 0, "msg": "success", "data": {"task": [{"jobID": "job_boss_test-job-id", "taskType": "batchTouch", "helloSum": 49, "timeMount": []}]}}'

## candidate filter

In [29]:
boss_candidate_info = json.loads(open('test_candidate.json').read())
r = requests.post(url+api_dict['filter_candidate'],json={
    'accountID': account_id,
    'jobID': job_id,
    'candidateInfo': boss_candidate_info,
    
})
r.text

'{"ret": 0, "msg": "success", "data": {"touch": false}}'

In [30]:
boss_candidate_info = json.loads(open('test_candidate_touch.json').read())
r = requests.post(url+api_dict['filter_candidate'],json={
    'accountID': account_id,
    'jobID': job_id,
    'candidateInfo': boss_candidate_info,
    
})
r.text

'{"ret": 0, "msg": "success", "data": {"touch": true}}'

In [ ]:
boss_candidate_info = json.loads(open('test_candidate_new.json.jso').read())
r = requests.post(url+api_dict['filter_candidate'],json={
    'accountID': account_id,
    'jobID': job_id,
    'candidateInfo': boss_candidate_info,
    
})
r.text

## chat

### search chat

In [38]:
candidate_id='mock-589791359'
candidate_name=candidate_id
history=[{'speaker': 'robot','msg':'这里是一家很牛逼的企业，看上你了，来吧老弟'}, {'speaker':'user','msg':'滚'}]
r = requests.post(url+api_dict['chat_candidate'],json={
    'accountID': account_id,
    'jobID': job_id,
    'candidateID': candidate_id,
    'candidateName': candidate_name,
    'historyMsg': history
})
r.text

'{"ret": 0, "msg": "success", "data": {"nextStep": "normal_chat", "nextStepContent": ""}}'

### user ask chat

In [39]:
candidate_id='mock-589791359'
candidate_name=candidate_id
history=[{'speaker': 'user','msg':'你们这里上班月薪有100万么'}]
r = requests.post(url+api_dict['chat_candidate'],json={
    'accountID': account_id,
    'jobID': job_id,
    'candidateID': candidate_id,
    'candidateName': candidate_name,
    'historyMsg': history
})
r.text

'{"ret": 0, "msg": "success", "data": {"nextStep": "normal_chat", "nextStepContent": "\\u4e0d\\u597d\\u610f\\u601d\\uff0c\\u6211\\u4eec\\u516c\\u53f8\\u7684\\u5ba2\\u670d\\u4e13\\u5458\\u5c97\\u4f4d\\u7684\\u7efc\\u5408\\u85aa\\u8d44\\u8303\\u56f4\\u662f4000-7000\\u5143\\u3002"}}'

## chat result

In [41]:
candidate_id='mock-589791359'
candidate_name=candidate_id
files = {'cv': open('test_cv.pdf','rb')}
r = requests.post(url+api_dict['result_candidate'], data={
    'accountID': account_id,
    'jobID': job_id,
    'candidateID': candidate_id,
    'candidateName': candidate_name,
    'phone': '139xxxx5556',
    'wechat': '139xxxx5556'
},files=files)
r.text


'{"ret": 0, "msg": "success", "data": {"status": "ok"}}'